<a href="https://colab.research.google.com/github/pavankosana/Detecting-Fake-Hindi-Tweets-on-CoVID-19-/blob/main/BERT_Fake_News_Using_BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

     |████████████████████████████████| 247 kB 4.2 MB/s 
     |████████████████████████████████| 9.1 MB 40.0 MB/s 
     |████████████████████████████████| 1.7 MB 41.7 MB/s 
     |████████████████████████████████| 1.2 MB 41.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 290 kB 45.0 MB/s 
     |████████████████████████████████| 3.1 MB 29.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 49.6 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 97 kB 7.1 MB/s 
     |████████████████████████████████| 180 kB 47.2 MB/s 
     |████████████████████████████████| 139 kB 50.4 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 243 kB 56.7 MB/s 
     |████████████████████████████████| 132 kB 51.2 MB/s 
     |█████████████████

simpletransformers==0.63.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as  pd


#Preprocessing Hindi true-fake News

In [3]:
import pandas as pd


In [4]:
df_fake_news_translated = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project_shared/fakeNews_translated (1).xlsx")
df_true_news_translated  = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project_shared/trueNews_translated (1).xlsx")
df_fake_news_translated.head()

,Date Posted,Link,Text,Region,Country,Explanation,Origin,Origin_URL,Fact_checked_by,Poynter_Label,Binary Label,translated_tweet
0,2020-07-02 00:00:00,https://www.poynter.org/?ifcn_misinformation=t...,Tencent revealed the real number of deaths.\t\t,Europe,France,The screenshot is questionable.,Twitter,https://www.liberation.fr/checknews/2020/02/07...,CheckNews,Misleading,0,सीरम इंस्टीट्यूट ऑफ इंडिया ने हाल ही में घोषणा...
1,2020-07-02 00:00:00,https://www.poynter.org/?ifcn_misinformation=t...,Taking chlorine dioxide helps fight coronavir...,Europe,Germany,Chlorine dioxide does guard against the coron...,Website,https://correctiv.org/faktencheck/medizin-und-...,Correctiv,False,0,"एक जहाज के दो चीनी चालक दल के सदस्य, जो COVID-..."
2,2020-07-02 00:00:00,https://www.poynter.org/?ifcn_misinformation=t...,This video shows workmen uncovering a bat-inf...,India,India,A video shows bats nesting in the roof; howev...,Facebook,https://factcheck.afp.com/video-shows-workmen-...,AFP,MISLEADING,0,"डायमंड प्रिंसेस के मूल 3,711 लोगों में से अब त..."
3,2020-07-02 00:00:00,https://www.poynter.org/?ifcn_misinformation=t...,The Asterix comic books and The Simpsons pred...,India,India,Coronavirus has been around since the 1960s. ...,Twitter,https://www.boomlive.in/health/did-the-simpson...,BOOM FactCheck,Misleading,0,#कोविड-19 | अब तक #SouthKorea में वायरस के कुल...
4,2020-07-02 00:00:00,https://www.poynter.org/?ifcn_misinformation=c...,Chinese President Xi Jinping visited a mosque...,India,India,Chinese President Xi Jinping's visit to the m...,Facebook,http://newsmobile.in/articles/2020/02/07/chine...,NewsMobile,False,0,COVID-19 #कोरोनावायरस के प्रसार के मद्देनजर था...


In [5]:
def preprocess_tweet(tweet):

    tweet = tweet.lower()
    url = r'http\S+'
    tweet = re.sub(url, 'URL', tweet, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    tweet =  emoji.sub(r'', tweet)
    tweet = ' '.join([word[1:] if word[0] == '#' else word for word in tweet.split()])
    
    return tweet

In [6]:
def too_much_cleaned_tweet(tweet):

    #'covid','covid19','corona','coronavirus','0','1','2','3','4','5','6','7','8','9'
    cleaned_text = tweet.split()
    exclude_char_list = ['#','@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL']
    tweet_without_special_chars = [word for word in cleaned_text if all(ch not in word for ch in exclude_char_list)]
    tweet_without_special_chars = ' '.join(tweet_without_special_chars)
 
    return tweet_without_special_chars

    


In [10]:
list = []
import re

for i in range(len(df_true_news_translated)):
    df_true_news_translated.at[i,'translated_tweet'] = preprocess_tweet(df_true_news_translated.iloc[i]['translated_tweet'])

for tweet_text in df_true_news_translated['translated_tweet']:
    list.append(tweet_text)   #append(too_much_cleaned_tweet(tweet_text))

df_true_news_temp = pd.DataFrame(list,columns = ['too_much_cleaned_tweet'])

# df_true_news_temp = pd.DataFrame(df_true_news_translated['translated_tweet'],columns = ['too_much_cleaned_tweet'])

df_true_news_translated = pd.concat([df_true_news_translated,df_true_news_temp],axis=1)
df_true_news_temp

,too_much_cleaned_tweet
0,जस्ट इन: नोवेल कोरोनावायरस का नाम 'कोविड -19':...
1,who ने आधिकारिक तौर पर coronavirus को covid-19...
2,"un स्वास्थ्य एजेंसी ने घोषणा की कि ""covid-19"" ..."
3,टोक्यो में भारतीय दूतावास ने कहा है कि डायमंड ...
4,ग्राउंड जीरो | संभावित covid-19 प्रसार की तैया...
...,...
3788,कोरोनावायरस | कर्नाटक में स्वास्थ्य अधिकारियों...
3789,नेपाल ने देश के सभी पहाड़ों पर चढ़ाई रोक दी है...
3790,जैसे-जैसे बड़ी संख्या में देश स्कूल और विश्ववि...
3791,"कालाबुरागी के उपायुक्त श्री शरत के अनुसार, 31 ..."


In [11]:
df_true_news_temp['label']=1
df_true_news_temp

,too_much_cleaned_tweet,label
0,जस्ट इन: नोवेल कोरोनावायरस का नाम 'कोविड -19':...,1
1,who ने आधिकारिक तौर पर coronavirus को covid-19...,1
2,"un स्वास्थ्य एजेंसी ने घोषणा की कि ""covid-19"" ...",1
3,टोक्यो में भारतीय दूतावास ने कहा है कि डायमंड ...,1
4,ग्राउंड जीरो | संभावित covid-19 प्रसार की तैया...,1
...,...,...
3788,कोरोनावायरस | कर्नाटक में स्वास्थ्य अधिकारियों...,1
3789,नेपाल ने देश के सभी पहाड़ों पर चढ़ाई रोक दी है...,1
3790,जैसे-जैसे बड़ी संख्या में देश स्कूल और विश्ववि...,1
3791,"कालाबुरागी के उपायुक्त श्री शरत के अनुसार, 31 ...",1


In [12]:
list = []
import re

for i in range(len(df_fake_news_translated)):
    df_fake_news_translated.at[i,'translated_tweet'] = preprocess_tweet(df_fake_news_translated.iloc[i]['translated_tweet'])

for tweet_text in df_fake_news_translated['translated_tweet']:
    list.append(tweet_text)    #append(too_much_cleaned_tweet(tweet_text))

df_fake_news_temp = pd.DataFrame(list,columns = ['too_much_cleaned_tweet'])

df_fake_news_translated = pd.concat([df_fake_news_translated,df_fake_news_temp],axis=1)
df_fake_news_temp

,too_much_cleaned_tweet
0,सीरम इंस्टीट्यूट ऑफ इंडिया ने हाल ही में घोषणा...
1,"एक जहाज के दो चीनी चालक दल के सदस्य, जो covid-..."
2,"डायमंड प्रिंसेस के मूल 3,711 लोगों में से अब त..."
3,कोविड-19 | अब तक southkorea में वायरस के कुल 8...
4,covid-19 कोरोनावायरस के प्रसार के मद्देनजर थाई...
...,...
3790,जैसे-जैसे बड़ी संख्या में देश स्कूल और विश्ववि...
3791,"कालाबुरागी के उपायुक्त श्री शरत के अनुसार, 31 ..."
3792,"अब, तेलंगाना स्वास्थ्य विभाग के अधिकारियों पर ..."
3793,"नेशनल बास्केटबॉल एसोसिएशन ने अपने सीज़न को ""अग..."


In [13]:
df_fake_news_temp['label']=0
type(df_fake_news_temp['label'][5])
#df_fake_news_temp

numpy.int64

In [14]:
df_final=df_true_news_temp.append(df_fake_news_temp)
df_final

,too_much_cleaned_tweet,label
0,जस्ट इन: नोवेल कोरोनावायरस का नाम 'कोविड -19':...,1
1,who ने आधिकारिक तौर पर coronavirus को covid-19...,1
2,"un स्वास्थ्य एजेंसी ने घोषणा की कि ""covid-19"" ...",1
3,टोक्यो में भारतीय दूतावास ने कहा है कि डायमंड ...,1
4,ग्राउंड जीरो | संभावित covid-19 प्रसार की तैया...,1
...,...,...
3790,जैसे-जैसे बड़ी संख्या में देश स्कूल और विश्ववि...,0
3791,"कालाबुरागी के उपायुक्त श्री शरत के अनुसार, 31 ...",0
3792,"अब, तेलंगाना स्वास्थ्य विभाग के अधिकारियों पर ...",0
3793,"नेशनल बास्केटबॉल एसोसिएशन ने अपने सीज़न को ""अग...",0


In [15]:
df_final = df_final.sample(frac=1)
df_final

,too_much_cleaned_tweet,label
1250,दुनिया भर में 1952 नए covid-19 मामले मंगलवार क...,0
582,तुमकुरु में मरने वाले एक 65 वर्षीय व्यक्ति का ...,0
833,"""मुझे अभी जाने की आवश्यकता है"": अमेरिकी छात्र ...",0
636,कोरोनावायरस | बेंगलुरु के 24 वर्षीय तकनीकी विश...,1
1083,लंदन का एक्सेल सेंटर 'दिनों के भीतर' कोविड -19...,0
...,...,...
1844,"सेंसेक्स सोमवार को 153.27 अंक गिरकर 38,144.02 ...",1
1676,"""415 पुष्ट मामले अब तक रिपोर्ट किए गए, 23 मरीज...",0
841,स्वास्थ्य विशेषज्ञ कोविड -19 के लिए इबुप्रोफेन...,1
3359,संयुक्त राष्ट्र ने कहा कि covid-19 महामारी वैश...,1


# BERT Model

In [16]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_final, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (6070, 2)
test shape:  (1518, 2)


In [17]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
            
             "num_train_epochs": 1}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-uncased", # "distilbert-base-multilingual-cased",
    num_labels=2,
    args=train_args
)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [18]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/6070 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/759 [00:00<?, ?it/s]

(759, 0.34253591089294694)

In [19]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1518 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/190 [00:00<?, ?it/s]

{'mcc': 0.7729549056064302,
 'tp': 564,
 'tn': 763,
 'fp': 0,
 'fn': 191,
 'auroc': 0.942471769678769,
 'auprc': 0.9540363751728496,
 'f1': 0.8741765480895916,
 'acc': 0.8741765480895916,
 'eval_loss': 0.2767117255489881}

#Arabic dataset

In [ ]:
df_fake_news_translated = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project_shared/fakeNews_translated (1).xlsx")
df_true_news_translated  = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project_shared/trueNews_translated (1).xlsx")
df_fake_news_translated.head()

In [ ]:
df_news = pd.read_csv("/content/drive/MyDrive/sharesd/FakeNewsDetection-main/FakeNewsDetection-main/Datasets/ManualAnnotatedFakeNewsDataset.csv")
del df_news['Unnamed: 0']
df_news.sample(5)

,Text,labels
546,مبادرة انسانية رونالدو يحول فنادقه الى مستشفي...,0
70,هذا امر مذهل للغاية لان تمرين محاكاة الوباء لف...,0
1328,الصانداي تليغراف نشرت تقريرا لمراسلة الشؤون ال...,1
141,هل ستعذر من مسلمي الاويغور بعد طمس هويتهم او م...,0
399,بالفيديو رفع اذان الجمعة بمكبرات الصوت في الما...,0


In [ ]:
list(df_news)

['Text', 'labels']

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_news, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (1230, 2)
test shape:  (308, 2)


In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
            
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-uncased", # "distilbert-base-multilingual-cased",
    num_labels=2,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1230 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/154 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/154 [00:00<?, ?it/s]

(616, 0.4480803737329174)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/308 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/39 [00:00<?, ?it/s]

{'mcc': 0.5045088988051695,
 'tp': 90,
 'tn': 144,
 'fp': 35,
 'fn': 39,
 'auroc': 0.8333116798752761,
 'auprc': 0.7820819226335757,
 'f1': 0.7597402597402597,
 'acc': 0.7597402597402597,
 'eval_loss': 0.9345586526756867}

#English Dataset

In [ ]:
df_eng = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project/fake_new_dataset.xlsx")
df_eng.sample(5)

,Unnamed: 0,title,text,subcategory,label
1142,1026,No Pinoys in new confirmed 2019-nCoV cases on ...,No Filipino was among the 10 new confirmed 201...,true,1
422,306,Corona virus: Markets send recession signals,"The most important reference price for oil, th...",true,1
900,784,Moody’s Analytics: Deadly Corona Virus May Be ...,A man walks by an electronic stock board of a ...,partially false,0
2132,2016,Russia: subjecting participants to meetings wi...,The Kremlin announced Thursday that it has beg...,true,1
844,728,novel coronavirus: Union Health Secy directs s...,"As per WHO update, at least 41 cases of novel ...",true,1


In [ ]:
df_eng["Text"] = df_eng["title"]+df_eng["text"]
# del df_eng["title"]
# del df_eng["text"]
# del df_eng["subcategory"]
# del df_eng["Unnamed: 0"]
df_temp = pd.DataFrame()
df_temp["Text"] = df_eng["Text"]
df_temp["labels"] = df_eng["label"]
df_eng = df_temp
df_eng.sample(5)

In [ ]:
df_eng.to_excel("/content/drive/MyDrive/Hindi Tweets Project/2col_fake_new_dataset.xlsx")

#BERT on English Dataset

In [ ]:
df_eng = pd.read_excel("/content/drive/MyDrive/Hindi Tweets Project/2col_fake_new_dataset.xlsx")
del df_eng["Unnamed: 0"]
df_eng.head()

,Text,label
0,FACEBOOK DELETES MICHIGAN ANTI-LOCKDOWN GROUP ...,0
1,Other Viewpoints: COVID-19 is worse than the f...,1
2,Bermuda's COVID-19 cases surpass 100The Minist...,1
3,Purdue University says students face 'close to...,0
4,THE HIGH COST OF LOCKING DOWN AMERICA: “WE’VE ...,0


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_eng, test_size=0.20)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (2495, 2)
test shape:  (624, 2)


In [ ]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
            
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased", # "distilbert-base-multilingual-cased",
    num_labels=2,
    args=train_args
)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
model.train_model(train_df)

  0%|          | 0/2495 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/312 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/312 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/312 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/312 [00:00<?, ?it/s]

(1248, 0.46488879438495645)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

result

  0%|          | 0/624 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/78 [00:00<?, ?it/s]

{'mcc': 0.5692531431546597,
 'tp': 383,
 'tn': 128,
 'fp': 70,
 'fn': 43,
 'auroc': 0.855621947171243,
 'auprc': 0.9108758176644453,
 'f1': 0.8189102564102565,
 'acc': 0.8189102564102564,
 'eval_loss': 0.48694702906486315}